## Technical Indicators List

All the indicators that https://pypi.org/project/stockstats/ supports

In [1]:
INDICATORS_LIST_ALL = ['high_5_sma','rsi','boll','macd','cr','wr','cci','tr','atr','dma','pdi','dx','adx','adxr','trix','tema','vr','mfi','vwma',
                   'chop','ppo','stochrsi','supertrend','aroon','close_75_z','ao'
                   ]

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
#%matplotlib inline
import mplfinance as mpf # matplot.finance to plot candlesticks
import random
import itertools # iterators for efficient looping: https://docs.python.org/3/library/itertools.html 

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader # a veces puede dar error al hacer fetch de la data porque no encuentra un stock o sus precios, o por la timezone
from finrl import config_tickers # config_tickers es una clase en la que se guardan en listas los nombres de cada stock para cada índice
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

/Users/carolinaalbamaruganrubio/miniforge3/envs/ai4finance/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2021-10-01'
TEST_START_DATE = '2021-10-01'
TEST_END_DATE = '2023-03-01'

In [4]:
stock = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Shape of DataFrame:  (97013, 8)


In [5]:
stock # this DF contains all the stock price history from the train start date to the test end date, for every firm in the DJ30

,date,open,high,low,close,volume,tic,day
0,2010-01-04,7.622500,7.660714,7.585000,6.470742,493729600,AAPL,0
1,2010-01-04,56.630001,57.869999,56.560001,41.200790,5277400,AMGN,0
2,2010-01-04,40.810001,41.099998,40.389999,33.090420,6894300,AXP,0
3,2010-01-04,55.720001,56.389999,54.799999,43.777538,6186700,BA,0
4,2010-01-04,57.650002,59.189999,57.509998,40.190224,7325600,CAT,0
...,...,...,...,...,...,...,...,...
97008,2023-02-28,482.670013,483.359985,473.920013,465.066803,3902100,UNH,1
97009,2023-02-28,220.000000,221.770004,219.500000,217.824539,5385400,V,1
97010,2023-02-28,38.700001,38.970001,38.549999,35.538383,16685300,VZ,1
97011,2023-02-28,35.480000,35.779999,35.320000,32.750980,8847000,WBA,1


In [6]:
stock.day.describe()

count    97013.000000
mean         2.024131
std          1.398530
min          0.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          4.000000
Name: day, dtype: float64

Stock symbols

In [7]:
tickers = stock.tic.unique()

We wanted to get:

In [8]:
print(np.reshape(config_tickers.DOW_30_TICKER, (len(config_tickers.DOW_30_TICKER))))
print("Number of firms in the index: ",len(config_tickers.DOW_30_TICKER))

['AXP' 'AMGN' 'AAPL' 'BA' 'CAT' 'CSCO' 'CVX' 'GS' 'HD' 'HON' 'IBM' 'INTC'
 'JNJ' 'KO' 'JPM' 'MCD' 'MMM' 'MRK' 'MSFT' 'NKE' 'PG' 'TRV' 'UNH' 'CRM'
 'VZ' 'V' 'WBA' 'WMT' 'DIS' 'DOW']
Number of firms in the index:  30


Creo que DOW es el DJIA, la media

But we got (because of the unavailable timezone or price data):

In [9]:
print(tickers) # stocks we have to train the agent
print("Number of firms downloaded: ",len(tickers))

['AAPL' 'AMGN' 'AXP' 'BA' 'CAT' 'CRM' 'CSCO' 'CVX' 'DIS' 'GS' 'HD' 'HON'
 'IBM' 'INTC' 'JNJ' 'JPM' 'KO' 'MCD' 'MMM' 'MRK' 'MSFT' 'NKE' 'PG' 'TRV'
 'UNH' 'V' 'VZ' 'WBA' 'WMT' 'DOW']
Number of firms downloaded:  30


## Agent 1-

with following technical indicators:

In [10]:
# let's first take 4 technical indicators randomly from the supported indicators list
INDICATORS = ['high_5_sma']

In [11]:
INDICATORS

['high_5_sma']

In [12]:
# creamos instancia de FeatureEngineer indicando la configuración deseada para hacer el preproceso
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True, # AUN NO SE QUE ES ESTO
                    use_turbulence=True, # measures extreme asset price fluctuation --> if the turbulence index reaches a pre-defined threshold, the agent will halt buying action and start selling the holding shares gradually
                    user_defined_feature = False) # MIRA A VER QUÉ ERA DEFINIRSE UNA FEATURE (creo que es )

processed = fe.preprocess_data(stock)

[*********************100%%**********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (3310, 8)


Successfully added vix
Successfully added turbulence index


Comentario: hay indicadores que pese a estar documentados como soportados en https://pypi.org/project/stockstats/, esta versión de código no los reconoce. Habrá que comprobar mediante más pruebas que todos los incluídos en la lista de todos los indicadores, estén soportados por esta versión. 

In [13]:
processed

,date,open,high,low,close,volume,tic,day,high_5_sma,vix,turbulence
0,2010-01-04,7.622500,7.660714,7.585000,6.470742,493729600,AAPL,0,7.660714,20.040001,0.000000
1,2010-01-04,56.630001,57.869999,56.560001,41.200790,5277400,AMGN,0,57.869999,20.040001,0.000000
2,2010-01-04,40.810001,41.099998,40.389999,33.090420,6894300,AXP,0,41.099998,20.040001,0.000000
3,2010-01-04,55.720001,56.389999,54.799999,43.777538,6186700,BA,0,56.389999,20.040001,0.000000
4,2010-01-04,57.650002,59.189999,57.509998,40.190224,7325600,CAT,0,59.189999,20.040001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
95985,2023-02-27,488.769989,490.940002,481.959991,472.278229,3006200,UNH,0,494.414001,20.950001,7.163833
95986,2023-02-27,220.729996,221.440002,219.339996,218.230591,4255300,V,0,221.592001,20.950001,7.163833
95987,2023-02-27,38.990002,39.150002,38.630001,35.602486,14210900,VZ,0,39.360000,20.950001,7.163833
95988,2023-02-27,36.049999,36.080002,35.270000,32.621929,5580100,WBA,0,36.282000,20.950001,7.163833


In order to allow the Agent to have a consistent data structure to work with, we need to ensure that each stock has the same data range. This is necessary because not all stocks have data for every trading day due to holidays, stock-specific trading suspensions, or newly listed stocks. Therefore, we will establish a common data interval for each stock with day granularity and fill with 0 the (the more NaNs we have, the worst the performance will be). --> We can check this effect with other Stock Indices with less missing stock prices.

Convert the stock column to list

In [14]:
list_ticker = processed["tic"].unique().tolist()
print(list_ticker)
len(list_ticker)

['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT']


29

Create a list from the minimum to the maximum date and with the same granularity

In [15]:
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
list_date

['2010-01-04',
 '2010-01-05',
 '2010-01-06',
 '2010-01-07',
 '2010-01-08',
 '2010-01-09',
 '2010-01-10',
 '2010-01-11',
 '2010-01-12',
 '2010-01-13',
 '2010-01-14',
 '2010-01-15',
 '2010-01-16',
 '2010-01-17',
 '2010-01-18',
 '2010-01-19',
 '2010-01-20',
 '2010-01-21',
 '2010-01-22',
 '2010-01-23',
 '2010-01-24',
 '2010-01-25',
 '2010-01-26',
 '2010-01-27',
 '2010-01-28',
 '2010-01-29',
 '2010-01-30',
 '2010-01-31',
 '2010-02-01',
 '2010-02-02',
 '2010-02-03',
 '2010-02-04',
 '2010-02-05',
 '2010-02-06',
 '2010-02-07',
 '2010-02-08',
 '2010-02-09',
 '2010-02-10',
 '2010-02-11',
 '2010-02-12',
 '2010-02-13',
 '2010-02-14',
 '2010-02-15',
 '2010-02-16',
 '2010-02-17',
 '2010-02-18',
 '2010-02-19',
 '2010-02-20',
 '2010-02-21',
 '2010-02-22',
 '2010-02-23',
 '2010-02-24',
 '2010-02-25',
 '2010-02-26',
 '2010-02-27',
 '2010-02-28',
 '2010-03-01',
 '2010-03-02',
 '2010-03-03',
 '2010-03-04',
 '2010-03-05',
 '2010-03-06',
 '2010-03-07',
 '2010-03-08',
 '2010-03-09',
 '2010-03-10',
 '2010-03-

Assign every day to each stock

In [16]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2010-01-04', 'AAPL'),
 ('2010-01-04', 'AMGN'),
 ('2010-01-04', 'AXP'),
 ('2010-01-04', 'BA'),
 ('2010-01-04', 'CAT'),
 ('2010-01-04', 'CRM'),
 ('2010-01-04', 'CSCO'),
 ('2010-01-04', 'CVX'),
 ('2010-01-04', 'DIS'),
 ('2010-01-04', 'GS'),
 ('2010-01-04', 'HD'),
 ('2010-01-04', 'HON'),
 ('2010-01-04', 'IBM'),
 ('2010-01-04', 'INTC'),
 ('2010-01-04', 'JNJ'),
 ('2010-01-04', 'JPM'),
 ('2010-01-04', 'KO'),
 ('2010-01-04', 'MCD'),
 ('2010-01-04', 'MMM'),
 ('2010-01-04', 'MRK'),
 ('2010-01-04', 'MSFT'),
 ('2010-01-04', 'NKE'),
 ('2010-01-04', 'PG'),
 ('2010-01-04', 'TRV'),
 ('2010-01-04', 'UNH'),
 ('2010-01-04', 'V'),
 ('2010-01-04', 'VZ'),
 ('2010-01-04', 'WBA'),
 ('2010-01-04', 'WMT'),
 ('2010-01-05', 'AAPL'),
 ('2010-01-05', 'AMGN'),
 ('2010-01-05', 'AXP'),
 ('2010-01-05', 'BA'),
 ('2010-01-05', 'CAT'),
 ('2010-01-05', 'CRM'),
 ('2010-01-05', 'CSCO'),
 ('2010-01-05', 'CVX'),
 ('2010-01-05', 'DIS'),
 ('2010-01-05', 'GS'),
 ('2010-01-05', 'HD'),
 ('2010-01-05', 'HON'),
 ('2010-01-05', 'IB

Merge with stock DF (potentially resulting in NaN values)

In [17]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")

Remove non-trading days generated with the date range

In [18]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]

In [19]:
processed_full

,date,tic,open,high,low,close,volume,day,high_5_sma,vix,turbulence
0,2010-01-04,AAPL,7.622500,7.660714,7.585000,6.470742,493729600.0,0.0,7.660714,20.040001,0.000000
1,2010-01-04,AMGN,56.630001,57.869999,56.560001,41.200790,5277400.0,0.0,57.869999,20.040001,0.000000
2,2010-01-04,AXP,40.810001,41.099998,40.389999,33.090420,6894300.0,0.0,41.099998,20.040001,0.000000
3,2010-01-04,BA,55.720001,56.389999,54.799999,43.777538,6186700.0,0.0,56.389999,20.040001,0.000000
4,2010-01-04,CAT,57.650002,59.189999,57.509998,40.190224,7325600.0,0.0,59.189999,20.040001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
139282,2023-02-27,UNH,488.769989,490.940002,481.959991,472.278229,3006200.0,0.0,494.414001,20.950001,7.163833
139283,2023-02-27,V,220.729996,221.440002,219.339996,218.230591,4255300.0,0.0,221.592001,20.950001,7.163833
139284,2023-02-27,VZ,38.990002,39.150002,38.630001,35.602486,14210900.0,0.0,39.360000,20.950001,7.163833
139285,2023-02-27,WBA,36.049999,36.080002,35.270000,32.621929,5580100.0,0.0,36.282000,20.950001,7.163833


In [20]:
len(processed_full.tic.unique())

29

In [21]:
processed_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95990 entries, 0 to 139286
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        95990 non-null  object 
 1   tic         95990 non-null  object 
 2   open        95990 non-null  float64
 3   high        95990 non-null  float64
 4   low         95990 non-null  float64
 5   close       95990 non-null  float64
 6   volume      95990 non-null  float64
 7   day         95990 non-null  float64
 8   high_5_sma  95990 non-null  float64
 9   vix         95990 non-null  float64
 10  turbulence  95990 non-null  float64
dtypes: float64(9), object(2)
memory usage: 8.8+ MB


In [22]:
processed_full.describe()

,open,high,low,close,volume,day,high_5_sma,vix,turbulence
count,95990.000000,95990.000000,95990.000000,95990.000000,9.599000e+04,95990.000000,95990.000000,95990.000000,95990.000000
mean,102.224117,103.175365,101.253646,88.243726,1.995383e+07,2.024471,103.102538,18.721163,34.943607
std,73.520545,74.279832,72.738958,70.607970,6.204278e+07,1.398649,74.216049,7.269653,43.065955
min,6.870357,7.000000,6.794643,5.806766,3.054000e+05,0.000000,7.045357,9.140000,0.000000
25%,47.470001,47.910000,47.020000,38.935156,4.003909e+06,1.000000,47.854000,13.630000,14.972915
50%,83.498051,84.139999,82.830002,67.654850,7.250700e+06,2.000000,84.121000,16.875000,24.175797
75%,138.477531,139.731735,137.169998,118.501184,1.498672e+07,3.000000,139.601499,21.760000,40.000776
max,555.000000,558.099976,550.130005,540.800171,1.880998e+09,4.000000,553.847998,82.690002,652.614837


In this case, there are no nulls, but if there were we would need to fill them in the following way. 

In [23]:
processed_full = processed_full.sort_values(['date','tic']) # we need to sort bc of the data_split method CHECK SOURCE CODE 

processed_full = processed_full.fillna(0)

In [24]:
processed_full.describe()

,open,high,low,close,volume,day,high_5_sma,vix,turbulence
count,95990.000000,95990.000000,95990.000000,95990.000000,9.599000e+04,95990.000000,95990.000000,95990.000000,95990.000000
mean,102.224117,103.175365,101.253646,88.243726,1.995383e+07,2.024471,103.102538,18.721163,34.943607
std,73.520545,74.279832,72.738958,70.607970,6.204278e+07,1.398649,74.216049,7.269653,43.065955
min,6.870357,7.000000,6.794643,5.806766,3.054000e+05,0.000000,7.045357,9.140000,0.000000
25%,47.470001,47.910000,47.020000,38.935156,4.003909e+06,1.000000,47.854000,13.630000,14.972915
50%,83.498051,84.139999,82.830002,67.654850,7.250700e+06,2.000000,84.121000,16.875000,24.175797
75%,138.477531,139.731735,137.169998,118.501184,1.498672e+07,3.000000,139.601499,21.760000,40.000776
max,555.000000,558.099976,550.130005,540.800171,1.880998e+09,4.000000,553.847998,82.690002,652.614837


In [25]:
large_value = 1e9

processed_full.replace([np.inf], large_value, inplace=True)

In [26]:
processed_full.describe()

,open,high,low,close,volume,day,high_5_sma,vix,turbulence
count,95990.000000,95990.000000,95990.000000,95990.000000,9.599000e+04,95990.000000,95990.000000,95990.000000,95990.000000
mean,102.224117,103.175365,101.253646,88.243726,1.995383e+07,2.024471,103.102538,18.721163,34.943607
std,73.520545,74.279832,72.738958,70.607970,6.204278e+07,1.398649,74.216049,7.269653,43.065955
min,6.870357,7.000000,6.794643,5.806766,3.054000e+05,0.000000,7.045357,9.140000,0.000000
25%,47.470001,47.910000,47.020000,38.935156,4.003909e+06,1.000000,47.854000,13.630000,14.972915
50%,83.498051,84.139999,82.830002,67.654850,7.250700e+06,2.000000,84.121000,16.875000,24.175797
75%,138.477531,139.731735,137.169998,118.501184,1.498672e+07,3.000000,139.601499,21.760000,40.000776
max,555.000000,558.099976,550.130005,540.800171,1.880998e+09,4.000000,553.847998,82.690002,652.614837


Split Train and Test datasets

In [27]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
train

,date,tic,open,high,low,close,volume,day,high_5_sma,vix,turbulence
0,2010-01-04,AAPL,7.622500,7.660714,7.585000,6.470742,493729600.0,0.0,7.660714,20.040001,0.000000
0,2010-01-04,AMGN,56.630001,57.869999,56.560001,41.200790,5277400.0,0.0,57.869999,20.040001,0.000000
0,2010-01-04,AXP,40.810001,41.099998,40.389999,33.090420,6894300.0,0.0,41.099998,20.040001,0.000000
0,2010-01-04,BA,55.720001,56.389999,54.799999,43.777538,6186700.0,0.0,56.389999,20.040001,0.000000
0,2010-01-04,CAT,57.650002,59.189999,57.509998,40.190224,7325600.0,0.0,59.189999,20.040001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2956,2021-09-30,UNH,401.489990,403.489990,390.459991,375.759735,3779900.0,3.0,405.876001,23.139999,24.927608
2956,2021-09-30,V,227.580002,228.789993,222.630005,218.139236,7128500.0,3.0,230.391998,23.139999,24.927608
2956,2021-09-30,VZ,54.500000,54.509998,54.000000,45.622711,18736600.0,3.0,54.616000,23.139999,24.927608
2956,2021-09-30,WBA,48.790001,48.930000,46.919998,40.519115,6449400.0,3.0,49.336000,23.139999,24.927608


In [28]:
len(train.tic.unique())

29

In [29]:
train.tic.unique()

array(['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS',
       'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM',
       'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT'],
      dtype=object)

In [30]:
trade = data_split(processed_full, TEST_START_DATE,TEST_END_DATE)
trade

,date,tic,open,high,low,close,volume,day,high_5_sma,vix,turbulence
0,2021-10-01,AAPL,141.899994,142.919998,139.110001,140.653503,94639600.0,4.0,144.492001,21.100000,120.031298
0,2021-10-01,AMGN,213.589996,214.610001,210.800003,195.822250,2629400.0,4.0,215.557999,21.100000,120.031298
0,2021-10-01,AXP,168.500000,175.119995,168.479996,167.727463,3956000.0,4.0,175.300000,21.100000,120.031298
0,2021-10-01,BA,222.850006,226.720001,220.600006,226.000000,9113600.0,4.0,227.198001,21.100000,120.031298
0,2021-10-01,CAT,192.899994,195.869995,191.240005,183.514038,3695500.0,4.0,199.956000,21.100000,120.031298
...,...,...,...,...,...,...,...,...,...,...,...
352,2023-02-27,UNH,488.769989,490.940002,481.959991,472.278229,3006200.0,0.0,494.414001,20.950001,7.163833
352,2023-02-27,V,220.729996,221.440002,219.339996,218.230591,4255300.0,0.0,221.592001,20.950001,7.163833
352,2023-02-27,VZ,38.990002,39.150002,38.630001,35.602486,14210900.0,0.0,39.360000,20.950001,7.163833
352,2023-02-27,WBA,36.049999,36.080002,35.270000,32.621929,5580100.0,0.0,36.282000,20.950001,7.163833


In [31]:
train_length = len(train)
trade_length = len(trade)
print("Number of training samples: ", train_length)
print("Number of testing samples", trade_length)

Number of training samples:  85753
Number of testing samples 10237


In [32]:
train.tail()

,date,tic,open,high,low,close,volume,day,high_5_sma,vix,turbulence
2956,2021-09-30,UNH,401.489990,403.489990,390.459991,375.759735,3779900.0,3.0,405.876001,23.139999,24.927608
2956,2021-09-30,V,227.580002,228.789993,222.630005,218.139236,7128500.0,3.0,230.391998,23.139999,24.927608
2956,2021-09-30,VZ,54.500000,54.509998,54.000000,45.622711,18736600.0,3.0,54.616000,23.139999,24.927608
2956,2021-09-30,WBA,48.790001,48.930000,46.919998,40.519115,6449400.0,3.0,49.336000,23.139999,24.927608
2956,2021-09-30,WMT,46.880001,47.243332,46.416668,44.566536,22457700.0,3.0,47.524667,23.139999,24.927608


In [33]:
trade.head()

,date,tic,open,high,low,close,volume,day,high_5_sma,vix,turbulence
0,2021-10-01,AAPL,141.899994,142.919998,139.110001,140.653503,94639600.0,4.0,144.492001,21.1,120.031298
0,2021-10-01,AMGN,213.589996,214.610001,210.800003,195.822250,2629400.0,4.0,215.557999,21.1,120.031298
0,2021-10-01,AXP,168.500000,175.119995,168.479996,167.727463,3956000.0,4.0,175.300000,21.1,120.031298
0,2021-10-01,BA,222.850006,226.720001,220.600006,226.000000,9113600.0,4.0,227.198001,21.1,120.031298
0,2021-10-01,CAT,192.899994,195.869995,191.240005,183.514038,3695500.0,4.0,199.956000,21.1,120.031298


Since we need to set the parameters for the environment functions, we need to compute the stock dimension and state space.

In [34]:
# Number of unique stocks used for the training 
stock_dimension = len(train.tic.unique())
# {balance, close price, shares, N-technical indicators}
# Balance will occupy 1 input node, and it is computed as: balance = balance (t) − amount of money we pay to buy shares + amount of money we receive to sell shares
# We will have N input nodes for the stock prices and N additional input nodes to indicate the current number of shares for each stock --> 2*N
# we will have one node for every technical indicator for every stock --> M indicators * N stocks
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension (Number of different companies in which we want to invest initially): {stock_dimension}, State Space (Number of input nodes to feed to the network): {state_space}")

Stock Dimension (Number of different companies in which we want to invest initially): 29, State Space (Number of input nodes to feed to the network): 88


Most of the trading companies demand trading commissions or costs. Here, we will simulate a buying and selling commission of 0,1% for each transaction. We will assume the user starts having no shares of any stock. Let's also assume we have 1.000.000 dollars to invest with. We will also define a parameter that sets the maximum amount of shares to trade, h_max. It is interesting to set the scaling reward or gamma (importance we give to the future reward), so that we specify if we give more importance to the short or long term. We could modify these parameters, if needed.

In [35]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension
h_max = 100
initial_amount = 1000000
gamma = 1e-4 # we are assuming we want to get the highest return in the short term, preventing ourselves from the risk and uncertainty caused by external factors in the long term

HAZ PRUEBAS CON UN GAMMA MÁS ALTO

Let's set the configuration that will be passed to the Environment class

In [36]:
env_kwargs = {
    "hmax": h_max,
    "initial_amount": initial_amount, 
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": gamma
}

Let's create the Stock Trading Environment!

In [37]:
e_train_gym = StockTradingEnv(df = train, **env_kwargs) # creates instance of the Environment class
print("Environment class type", type(e_train_gym))
env_train, _ = e_train_gym.get_sb_env() # resets the environment and converts the initial environment into a DummyVecEnv instance
print("External Environment class type: ", type(env_train))

Environment class type <class 'finrl.meta.env_stock_trading.env_stocktrading.StockTradingEnv'>
External Environment class type:  <class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


Since we will use different RL algorithms, let's set some flags:

In [38]:
if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_td3 = True
if_using_sac = True

Let's create the Deep Reinforcement Learning Agent!

podríamos runear todos los ind (p.e. 3 o 4 ind) y con los mejores, cambiar el resto de parametros (fechas, gamma,...)

# A2C

In [39]:
agent = DRLAgent(env = env_train) # creates Agent instance
model_a2c = agent.get_model("a2c") # gets stablebaselines3 model  

if if_using_a2c: # where to store the results
  # set up logger
  tmp_path = 'resultadosTFG' + '/set_1-' + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to resultadosTFG/set_1-/a2c


Let's train the agent!

In [40]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

---------------------------------------
| time/                 |             |
|    fps                | 179         |
|    iterations         | 100         |
|    time_elapsed       | 2           |
|    total_timesteps    | 500         |
| train/                |             |
|    entropy_loss       | -41.3       |
|    explained_variance | -0.419      |
|    learning_rate      | 0.0007      |
|    n_updates          | 99          |
|    policy_loss        | -42.1       |
|    reward             | -0.23576832 |
|    std                | 1.01        |
|    value_loss         | 1.81        |
---------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 177       |
|    iterations         | 200       |
|    time_elapsed       | 5         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -41.3     |
|    explained_variance | 0.0266    |
|    learning_rate

In [41]:
trained_a2c.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=88, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=88, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=29, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

Let's see the learning progress of the agent

In [42]:
learning_process = pd.read_csv("resultadosTFG/set_1-/a2c/progress.csv")
learning_process

,time/total_timesteps,time/iterations,train/reward,train/std,train/entropy_loss,time/fps,train/value_loss,train/n_updates,train/learning_rate,train/policy_loss,time/time_elapsed,train/explained_variance
0,500,100,-0.235768,1.006123,-41.323139,179,1.813766,99,0.0007,-42.076424,2,-4.189264e-01
1,1000,200,1.312529,1.004437,-41.269428,177,5.446766,199,0.0007,-49.025402,5,2.655917e-02
2,1500,300,-1.836116,1.002080,-41.203060,177,7.670699,299,0.0007,-56.252846,8,5.960464e-08
3,2000,400,-2.326824,1.003625,-41.253872,180,19.977612,399,0.0007,-146.844711,11,1.045066e-02
4,2500,500,-1.187710,1.001414,-41.188374,180,6.804198,499,0.0007,41.596882,13,-1.192093e-07
...,...,...,...,...,...,...,...,...,...,...,...,...
95,48000,9600,0.093128,1.071335,-43.096603,168,0.238420,9599,0.0007,0.362389,285,5.960464e-08
96,48500,9700,-1.642893,1.073632,-43.157925,168,2.269016,9699,0.0007,59.487450,288,0.000000e+00
97,49000,9800,-1.033882,1.074037,-43.168339,168,31.598492,9799,0.0007,-252.488281,291,0.000000e+00
98,49500,9900,-2.045254,1.073565,-43.155228,168,3.109994,9899,0.0007,49.501465,294,0.000000e+00


Plot the training results

In [43]:
def plot_metrics(df, n_rows, n_cols):
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(5 * n_cols, 5 * n_rows))
    axes = axes.flatten()  # Flatten the array of axes if more than one row or column

    # List of tuples containing (y-axis, title)
    metrics = [
        ('train/reward', 'Reward'),
        ('train/policy_loss', 'Policy Loss'),
        ('train/std', 'Standard Deviation'),
        ('train/n_updates', 'Number of Updates'),
        ('train/value_loss', 'Value Loss'),
        ('time/fps', 'Frames Per Second'),
        ('train/explained_variance', 'Explained Variance'),
        ('train/entropy_loss', 'Entropy Loss')
    ]

    # Plot each metric
    for ax, (metric, title) in zip(axes, metrics):
        ax.plot(df['time/total_timesteps'], df[metric], label=title)
        ax.set_xlabel('Total Timesteps')
        ax.set_ylabel(title)
        ax.set_title(title)
        ax.grid(True)
        ax.legend()

    plt.tight_layout()
    plt.savefig(f'resultadosTFG/set_1-/a2c/trainingStatsResults.png')
    plt.close()

df = learning_process
plot_metrics(df, n_rows=4, n_cols=2)  # Modify n_rows and n_cols as needed


The agent is not learning very well... 

Podríamos hacer una tabla resumen de todos los algos y pones cuántos steps han sido necesarios, la LR de cada uno, etc

# PPO

Set PPO parameters and set the logger

In [44]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = 'resultadosTFG' + '/set_1-' + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device
Logging to resultadosTFG/set_1-/ppo


Train PPO

In [45]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000) if if_using_ppo else None

-----------------------------------
| time/              |            |
|    fps             | 214        |
|    iterations      | 1          |
|    time_elapsed    | 9          |
|    total_timesteps | 2048       |
| train/             |            |
|    reward          | 0.38917002 |
-----------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 212         |
|    iterations           | 2           |
|    time_elapsed         | 19          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.016276129 |
|    clip_fraction        | 0.232       |
|    clip_range           | 0.2         |
|    entropy_loss         | -41.2       |
|    explained_variance   | 0.0192      |
|    learning_rate        | 0.00025     |
|    loss                 | 4.99        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0335     |
|   

In [46]:
trained_ppo.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=88, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=88, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=29, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

Let's see the learning progress of the agent

In [47]:
learning_process = pd.read_csv("resultadosTFG/set_1-/ppo/progress.csv")
learning_process

,time/fps,time/total_timesteps,time/iterations,time/time_elapsed,train/reward,train/loss,train/clip_fraction,train/value_loss,train/explained_variance,train/clip_range,train/policy_gradient_loss,train/std,train/entropy_loss,train/approx_kl,train/n_updates,train/learning_rate
0,214,2048,1,9,0.389170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,212,4096,2,19,0.575630,4.989151,0.231836,11.440086,0.019246,0.2,-0.033527,1.003808,-41.210823,0.016276,10.0,0.00025
2,212,6144,3,28,2.216731,16.281906,0.145215,38.224171,-0.005132,0.2,-0.022372,1.005499,-41.279159,0.013840,20.0,0.00025
3,212,8192,4,38,4.047955,39.488770,0.161377,62.583490,0.002299,0.2,-0.021605,1.008465,-41.343777,0.014888,30.0,0.00025
4,212,10240,5,48,-1.491794,9.254026,0.194141,16.697088,-0.026347,0.2,-0.028568,1.011617,-41.432018,0.020798,40.0,0.00025
5,212,12288,6,57,0.271097,8.371807,0.149414,34.357820,-0.003534,0.2,-0.024073,1.013595,-41.507797,0.015491,50.0,0.00025
6,212,14336,7,67,1.159772,10.326603,0.148535,35.833716,-0.008125,0.2,-0.021758,1.014382,-41.550824,0.014427,60.0,0.00025
7,213,16384,8,76,-2.186234,14.887259,0.203320,18.926094,0.004241,0.2,-0.026837,1.017013,-41.597803,0.018512,70.0,0.00025
8,212,18432,9,86,0.916854,19.530544,0.242627,33.879628,-0.001761,0.2,-0.027301,1.019076,-41.660916,0.025289,80.0,0.00025
9,212,20480,10,96,-1.323506,16.463291,0.229980,39.847125,-0.022609,0.2,-0.019135,1.021770,-41.731975,0.020596,90.0,0.00025


Plot the training results

In [48]:
def plot_metrics(df, n_rows, n_cols):
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(5 * n_cols, 5 * n_rows))
    axes = axes.flatten()  # Flatten the array of axes if more than one row or column

    metrics = [
        ('train/reward', 'Reward'),
        ('train/loss', 'Loss'),
        ('train/std', 'Standard Deviation'),
        ('train/approx_kl', 'approximate Kullback-Leibler divergence'),
        ('train/clip_fraction', 'Clip Fraction'),
        ('train/clip_range', 'Clip Range'),
        ('train/explained_variance', 'Explained Variance'),
        ('train/value_loss', 'Value Loss'),
        ('train/policy_gradient_loss', 'Policy Gradient Loss'),
        ('train/entropy_loss', 'Entropy Loss')
    ]

    # Plot each metric
    for ax, (metric, title) in zip(axes, metrics):
        ax.plot(df['time/total_timesteps'], df[metric], label=title)
        ax.set_xlabel('Total Timesteps')
        ax.set_ylabel(title)
        ax.set_title(title)
        ax.grid(True)
        ax.legend()

    plt.tight_layout()
    plt.savefig(f'resultadosTFG/set_1-/ppo/trainingStatsResults.png')
    plt.close()

df = learning_process
plot_metrics(df, n_rows=5, n_cols=2)  # Modify n_rows and n_cols as needed

# DDPG

Create the agent and set the logger

In [49]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

if if_using_ddpg:
  # set up logger
  tmp_path = 'resultadosTFG' + '/set_1-' + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device
Logging to resultadosTFG/set_1-/ddpg


Train the agent

In [50]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000) if if_using_ddpg else None

day: 2956, episode: 40
begin_total_asset: 1000000.00
end_total_asset: 4543072.55
total_reward: 3543072.55
total_cost: 999.00
total_trades: 44274
Sharpe: 0.804
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 115       |
|    time_elapsed    | 102       |
|    total_timesteps | 11828     |
| train/             |           |
|    actor_loss      | -208      |
|    critic_loss     | 6.66e+03  |
|    learning_rate   | 0.001     |
|    n_updates       | 8871      |
|    reward          | -8.708863 |
----------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 107       |
|    time_elapsed    | 220       |
|    total_timesteps | 23656     |
| train/             |           |
|    actor_loss      | -140      |
|    critic_loss     | 42.3      |
|    learning_rate   | 0.001     |
|    n_updates       | 20699     |
|

In [51]:
trained_ddpg.policy

TD3Policy(
  (actor): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (mu): Sequential(
      (0): Linear(in_features=88, out_features=400, bias=True)
      (1): ReLU()
      (2): Linear(in_features=400, out_features=300, bias=True)
      (3): ReLU()
      (4): Linear(in_features=300, out_features=29, bias=True)
      (5): Tanh()
    )
  )
  (actor_target): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (mu): Sequential(
      (0): Linear(in_features=88, out_features=400, bias=True)
      (1): ReLU()
      (2): Linear(in_features=400, out_features=300, bias=True)
      (3): ReLU()
      (4): Linear(in_features=300, out_features=29, bias=True)
      (5): Tanh()
    )
  )
  (critic): ContinuousCritic(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (qf0): Sequential(
      (0): Linear(in_features=117, out_fe

Let's see the learning progress of the agent 

In [52]:
learning_process = pd.read_csv("resultadosTFG/set_1-/ddpg/progress.csv")
learning_process

,time/total_timesteps,train/actor_loss,train/reward,train/critic_loss,time/fps,train/n_updates,time/episodes,train/learning_rate,time/time_elapsed
0,11828,-208.465194,-8.708863,6661.996026,115,8871,4,0.001,102
1,23656,-139.999263,-8.708863,42.270130,107,20699,8,0.001,220
2,35484,-82.785095,-8.708863,7.620665,100,32527,12,0.001,351
3,47312,-52.103502,-8.708863,5.108380,99,44355,16,0.001,475


Plot the results of the training

In [53]:
def plot_metrics(df, n_rows, n_cols):
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(5 * n_cols, 5 * n_rows))
    axes = axes.flatten()  # Flatten the array of axes if more than one row or column

    metrics = [
        ('train/reward', 'Reward'),
        ('train/actor_loss', 'Actor Loss'),
        ('train/n_updates', 'Number of Updates'),
        ('train/critic_loss', 'Critic Loss')
    ]

    # Plot each metric
    for ax, (metric, title) in zip(axes, metrics):
        ax.plot(df['time/total_timesteps'], df[metric], label=title)
        ax.set_xlabel('Total Timesteps')
        ax.set_ylabel(title)
        ax.set_title(title)
        ax.grid(True)
        ax.legend()

    plt.tight_layout()
    plt.savefig(f'resultadosTFG/set_1-/ddpg/trainingStatsResults.png')
    plt.close()

df = learning_process
plot_metrics(df, n_rows=4, n_cols=1)  # Modify n_rows and n_cols as needed

# TD3

Create the agent and set the logger

In [54]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = 'resultadosTFG' + '/set_1-' + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device
Logging to resultadosTFG/set_1-/td3


Train TD3

In [55]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000) if if_using_td3 else None

----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 112       |
|    time_elapsed    | 105       |
|    total_timesteps | 11828     |
| train/             |           |
|    actor_loss      | 201       |
|    critic_loss     | 1.44e+04  |
|    learning_rate   | 0.001     |
|    n_updates       | 8871      |
|    reward          | -6.325187 |
----------------------------------
day: 2956, episode: 60
begin_total_asset: 1000000.00
end_total_asset: 3718121.86
total_reward: 2718121.86
total_cost: 998.99
total_trades: 38428
Sharpe: 0.682
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 104       |
|    time_elapsed    | 227       |
|    total_timesteps | 23656     |
| train/             |           |
|    actor_loss      | 194       |
|    critic_loss     | 3.24e+03  |
|    learning_rate   | 0.001     |
|    n_updates       | 20699     |
|

In [56]:
trained_td3.policy

TD3Policy(
  (actor): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (mu): Sequential(
      (0): Linear(in_features=88, out_features=400, bias=True)
      (1): ReLU()
      (2): Linear(in_features=400, out_features=300, bias=True)
      (3): ReLU()
      (4): Linear(in_features=300, out_features=29, bias=True)
      (5): Tanh()
    )
  )
  (actor_target): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (mu): Sequential(
      (0): Linear(in_features=88, out_features=400, bias=True)
      (1): ReLU()
      (2): Linear(in_features=400, out_features=300, bias=True)
      (3): ReLU()
      (4): Linear(in_features=300, out_features=29, bias=True)
      (5): Tanh()
    )
  )
  (critic): ContinuousCritic(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (qf0): Sequential(
      (0): Linear(in_features=117, out_fe

Let's see agent's learning process

In [57]:
learning_process = pd.read_csv("resultadosTFG/set_1-/td3/progress.csv")
learning_process

,time/total_timesteps,train/actor_loss,train/reward,train/critic_loss,time/fps,train/n_updates,time/episodes,train/learning_rate,time/time_elapsed
0,11828,201.150929,-6.325187,14354.700131,112,8871,4,0.001,105
1,23656,193.777964,-6.325187,3244.935202,104,20699,8,0.001,227
2,35484,139.126243,-6.325187,151.505362,101,32527,12,0.001,349
3,47312,104.101287,-6.325187,39.004947,100,44355,16,0.001,468


Plot training results

In [58]:
def plot_metrics(df, n_rows, n_cols):
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(5 * n_cols, 5 * n_rows))
    axes = axes.flatten()  # Flatten the array of axes if more than one row or column

    metrics = [
        ('train/reward', 'Reward'),
        ('train/actor_loss', 'Actor Loss'),
        ('train/n_updates', 'Number of Updates'),
        ('train/critic_loss', 'Critic Loss')
    ]

    # Plot each metric
    for ax, (metric, title) in zip(axes, metrics):
        ax.plot(df['time/total_timesteps'], df[metric], label=title)
        ax.set_xlabel('Total Timesteps')
        ax.set_ylabel(title)
        ax.set_title(title)
        ax.grid(True)
        ax.legend()

    plt.tight_layout()
    plt.savefig(f'resultadosTFG/set_1-/td3/trainingStatsResults.png')
    plt.close()

df = learning_process
plot_metrics(df, n_rows=4, n_cols=1)  # Modify n_rows and n_cols as needed

# SAC

Create the agent, set parameters for training, and set logger

In [59]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = 'resultadosTFG' + '/set_1-' + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device
Logging to resultadosTFG/set_1-/sac


Train SAC

In [60]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=50000) if if_using_sac else None

-----------------------------------
| time/              |            |
|    episodes        | 4          |
|    fps             | 69         |
|    time_elapsed    | 170        |
|    total_timesteps | 11828      |
| train/             |            |
|    actor_loss      | 1.79e+03   |
|    critic_loss     | 147        |
|    ent_coef        | 0.289      |
|    ent_coef_loss   | 153        |
|    learning_rate   | 0.0001     |
|    n_updates       | 11727      |
|    reward          | -8.7768755 |
-----------------------------------
day: 2956, episode: 80
begin_total_asset: 1000000.00
end_total_asset: 5997266.55
total_reward: 4997266.55
total_cost: 63746.44
total_trades: 62046
Sharpe: 0.857
-----------------------------------
| time/              |            |
|    episodes        | 8          |
|    fps             | 68         |
|    time_elapsed    | 347        |
|    total_timesteps | 23656      |
| train/             |            |
|    actor_loss      | 934        |
|    critic

In [61]:
trained_sac.policy

SACPolicy(
  (actor): Actor(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (latent_pi): Sequential(
      (0): Linear(in_features=88, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
    )
    (mu): Linear(in_features=256, out_features=29, bias=True)
    (log_std): Linear(in_features=256, out_features=29, bias=True)
  )
  (critic): ContinuousCritic(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (qf0): Sequential(
      (0): Linear(in_features=117, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=1, bias=True)
    )
    (qf1): Sequential(
      (0): Linear(in_features=117, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=25

Let's see agent's learning process

In [62]:
learning_process = pd.read_csv("resultadosTFG/set_1-/sac/progress.csv")
learning_process

,time/total_timesteps,train/ent_coef_loss,train/actor_loss,train/reward,train/critic_loss,time/fps,train/n_updates,train/ent_coef,time/episodes,train/learning_rate,time/time_elapsed
0,11828,153.402893,1792.315186,-8.776875,147.100952,69,11727,0.289364,4,0.0001,170
1,23656,-90.952209,934.442139,-2.309990,76.377739,68,23555,0.139960,8,0.0001,347
2,35484,-122.671478,448.741760,-7.824470,37.358887,68,35383,0.043510,12,0.0001,520
3,47312,-138.575943,225.406357,-8.480895,22.048031,67,47211,0.013598,16,0.0001,698


Agent's training results

In [63]:
def plot_metrics(df, n_rows, n_cols):
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(5 * n_cols, 5 * n_rows))
    axes = axes.flatten()  # Flatten the array of axes if more than one row or column

    metrics = [
        ('train/reward', 'Reward'),
        ('train/actor_loss', 'Actor Loss'),
        ('train/ent_coef_loss', 'Entropy coefficient Loss'),
        ('train/ent_coef', 'Entropy coefficient'),
        ('train/critic_loss', 'Critic Loss')
    ]

    # Plot each metric
    for ax, (metric, title) in zip(axes, metrics):
        ax.plot(df['time/total_timesteps'], df[metric], label=title)
        ax.set_xlabel('Total Timesteps')
        ax.set_ylabel(title)
        ax.set_title(title)
        ax.grid(True)
        ax.legend()

    plt.tight_layout()
    plt.savefig(f'resultadosTFG/set_1-/sac/trainingStatsResults.png')
    plt.close()

df = learning_process
plot_metrics(df, n_rows=5, n_cols=1)  # Modify n_rows and n_cols as needed

# Testing

There is a new environment for testing

We feed the environment with the testing dataset we prepared before (from start test date to end test date: with closing prices, etc)

In [64]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
e_trade_gym

Environment from Stable baselines3; observations is a matrix containing the inputs {balance, close price, shares, N-technical indicators}

In [65]:
env_trade, obs_trade = e_trade_gym.get_sb_env()
env_trade

In [66]:
obs_trade

array([[1.0000000e+06, 1.4065350e+02, 1.9582225e+02, 1.6772746e+02,
        2.2600000e+02, 1.8351404e+02, 2.7490091e+02, 5.0711670e+01,
        9.3807129e+01, 1.7543822e+02, 3.5233066e+02, 3.0777103e+02,
        2.0362247e+02, 1.2042436e+02, 5.0125824e+01, 1.4843318e+02,
        1.5452452e+02, 4.8765842e+01, 2.2827776e+02, 1.2634029e+02,
        7.5069099e+01, 2.8285742e+02, 1.4231578e+02, 1.3033842e+02,
        1.4367572e+02, 3.7738495e+02, 2.2568965e+02, 4.5867676e+01,
        4.0303814e+01, 4.3821529e+01, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e

In [67]:
trade

,date,tic,open,high,low,close,volume,day,high_5_sma,vix,turbulence
0,2021-10-01,AAPL,141.899994,142.919998,139.110001,140.653503,94639600.0,4.0,144.492001,21.100000,120.031298
0,2021-10-01,AMGN,213.589996,214.610001,210.800003,195.822250,2629400.0,4.0,215.557999,21.100000,120.031298
0,2021-10-01,AXP,168.500000,175.119995,168.479996,167.727463,3956000.0,4.0,175.300000,21.100000,120.031298
0,2021-10-01,BA,222.850006,226.720001,220.600006,226.000000,9113600.0,4.0,227.198001,21.100000,120.031298
0,2021-10-01,CAT,192.899994,195.869995,191.240005,183.514038,3695500.0,4.0,199.956000,21.100000,120.031298
...,...,...,...,...,...,...,...,...,...,...,...
352,2023-02-27,UNH,488.769989,490.940002,481.959991,472.278229,3006200.0,0.0,494.414001,20.950001,7.163833
352,2023-02-27,V,220.729996,221.440002,219.339996,218.230591,4255300.0,0.0,221.592001,20.950001,7.163833
352,2023-02-27,VZ,38.990002,39.150002,38.630001,35.602486,14210900.0,0.0,39.360000,20.950001,7.163833
352,2023-02-27,WBA,36.049999,36.080002,35.270000,32.621929,5580100.0,0.0,36.282000,20.950001,7.163833


## A2C

Test, predict

In [68]:
trained_model = trained_a2c
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_model, 
    environment = e_trade_gym)

el modelo es 
estos son los test obs
[[1.0000000e+06 1.4065350e+02 1.9582225e+02 1.6772746e+02 2.2600000e+02
  1.8351404e+02 2.7490091e+02 5.0711670e+01 9.3807129e+01 1.7543822e+02
  3.5233066e+02 3.0777103e+02 2.0362247e+02 1.2042436e+02 5.0125824e+01
  1.4843318e+02 1.5452452e+02 4.8765842e+01 2.2827776e+02 1.2634029e+02
  7.5069099e+01 2.8285742e+02 1.4231578e+02 1.3033842e+02 1.4367572e+02
  3.7738495e+02 2.2568965e+02 4.5867676e+01 4.0303814e+01 4.3821529e+01
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.4449200e+02
  2.1555800e+02 1.7530000e+02 2.2719800e+02 1.9995599e+02 2.7714801e+02
  5.5990002e+01 1.0418000e+

Plot and Save testing results, Portfolio Value

In [69]:
df_account_value_a2c['date'] = pd.to_datetime(df_account_value_a2c['date']).dt.normalize()
plt.figure(figsize=(14, 7))
plt.plot(df_account_value_a2c['date'], df_account_value_a2c['account_value'])

# Setting major locator
locator = mdates.MonthLocator(interval=1)  # Show a tick every week
plt.gca().xaxis.set_major_locator(locator)

# Setting formatter
formatter = mdates.DateFormatter('%Y-%m-%d')
plt.gca().xaxis.set_major_formatter(formatter)

# Optionally, auto-format date labels to prevent overlap
plt.gcf().autofmt_xdate()

plt.title('Account Value Testing for A2C')
plt.xlabel('Date')
plt.ylabel('Account Value')
plt.grid(True)

plt.savefig(f'resultadosTFG/set_1-/a2c/testing/accountValue.png')
plt.close()

Trades made in testing

In [70]:
display(df_actions_a2c)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2021-10-01,100,100,0,100,91,83,0,95,75,0,...,68,45,100,100,4,11,0,55,0,0
2021-10-04,100,100,0,100,91,83,0,95,75,0,...,68,45,100,100,4,11,0,55,0,0
2021-10-05,100,100,0,100,91,83,0,95,75,0,...,68,45,100,100,4,11,0,55,0,0
2021-10-06,100,100,0,100,91,83,0,95,75,0,...,68,45,100,100,4,11,0,55,0,0
2021-10-07,100,100,0,100,83,0,0,90,0,0,...,1,0,100,100,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-21,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## PPO

Test, predict

In [71]:
trained_model = trained_ppo
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_model, 
    environment = e_trade_gym)

el modelo es 
estos son los test obs
[[1.0000000e+06 1.4065350e+02 1.9582225e+02 1.6772746e+02 2.2600000e+02
  1.8351404e+02 2.7490091e+02 5.0711670e+01 9.3807129e+01 1.7543822e+02
  3.5233066e+02 3.0777103e+02 2.0362247e+02 1.2042436e+02 5.0125824e+01
  1.4843318e+02 1.5452452e+02 4.8765842e+01 2.2827776e+02 1.2634029e+02
  7.5069099e+01 2.8285742e+02 1.4231578e+02 1.3033842e+02 1.4367572e+02
  3.7738495e+02 2.2568965e+02 4.5867676e+01 4.0303814e+01 4.3821529e+01
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.4449200e+02
  2.1555800e+02 1.7530000e+02 2.2719800e+02 1.9995599e+02 2.7714801e+02
  5.5990002e+01 1.0418000e+

Save Plot of Portfolio Value

In [72]:
df_account_value_ppo['date'] = pd.to_datetime(df_account_value_ppo['date']).dt.normalize()
plt.figure(figsize=(14, 7))
plt.plot(df_account_value_ppo['date'], df_account_value_ppo['account_value'])

# Setting major locator
locator = mdates.MonthLocator(interval=1)  # Show a tick every week
plt.gca().xaxis.set_major_locator(locator)

# Setting formatter
formatter = mdates.DateFormatter('%Y-%m-%d')
plt.gca().xaxis.set_major_formatter(formatter)

# Optionally, auto-format date labels to prevent overlap
plt.gcf().autofmt_xdate()

plt.title('Account Value Testing for PPO')
plt.xlabel('Date')
plt.ylabel('Account Value')
plt.grid(True)

plt.savefig(f'resultadosTFG/set_1-/ppo/testing/accountValue.png')
plt.close()


Trades made in testing

In [73]:
df_actions_ppo.tail(40)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2022-12-28,0,0,19,0,7,12,8,0,0,-8,...,0,0,-3,-7,0,0,0,7,0,-1
2022-12-29,0,0,19,0,7,12,8,0,0,-8,...,0,0,-3,-7,0,0,0,7,0,-1
2022-12-30,0,0,19,0,2,12,2,0,0,-8,...,0,0,-6,-5,0,0,0,0,0,0
2023-01-03,0,0,19,0,0,10,0,0,0,-8,...,0,0,-5,-6,0,0,0,0,0,0
2023-01-04,0,0,19,0,0,9,0,0,0,-8,...,0,0,-3,-7,0,0,0,1,0,-1
2023-01-05,0,0,18,0,0,0,0,0,0,-5,...,0,0,-1,-6,0,0,0,1,0,0
2023-01-06,0,0,18,0,0,0,0,0,0,-5,...,0,0,-1,-6,0,0,0,0,0,0
2023-01-09,0,0,18,0,0,0,0,0,0,-5,...,0,0,-1,-6,0,0,0,1,0,0
2023-01-10,0,0,18,0,0,0,0,0,0,-5,...,0,0,-1,-6,0,0,0,0,0,0


## DDPG

Test DDPG, predict

In [74]:
trained_model = trained_ddpg
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_model, 
    environment = e_trade_gym)

el modelo es 
estos son los test obs
[[1.0000000e+06 1.4065350e+02 1.9582225e+02 1.6772746e+02 2.2600000e+02
  1.8351404e+02 2.7490091e+02 5.0711670e+01 9.3807129e+01 1.7543822e+02
  3.5233066e+02 3.0777103e+02 2.0362247e+02 1.2042436e+02 5.0125824e+01
  1.4843318e+02 1.5452452e+02 4.8765842e+01 2.2827776e+02 1.2634029e+02
  7.5069099e+01 2.8285742e+02 1.4231578e+02 1.3033842e+02 1.4367572e+02
  3.7738495e+02 2.2568965e+02 4.5867676e+01 4.0303814e+01 4.3821529e+01
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.4449200e+02
  2.1555800e+02 1.7530000e+02 2.2719800e+02 1.9995599e+02 2.7714801e+02
  5.5990002e+01 1.0418000e+

Store graph of the value of portfolio

In [75]:
df_account_value_ddpg['date'] = pd.to_datetime(df_account_value_ddpg['date']).dt.normalize()
plt.figure(figsize=(14, 7))
plt.plot(df_account_value_ddpg['date'], df_account_value_ddpg['account_value'])

# Setting major locator
locator = mdates.MonthLocator(interval=1)  # Show a tick every week
plt.gca().xaxis.set_major_locator(locator)

# Setting formatter
formatter = mdates.DateFormatter('%Y-%m-%d')
plt.gca().xaxis.set_major_formatter(formatter)

# Optionally, auto-format date labels to prevent overlap
plt.gcf().autofmt_xdate()

plt.title('Account Value Testing for DDPG')
plt.xlabel('Date')
plt.ylabel('Account Value')
plt.grid(True)

plt.savefig(f'resultadosTFG/set_1-/ddpg/testing/accountValue.png')
plt.close()


Trades made in testing

In [76]:
df_actions_ddpg.head(30)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2021-10-01,0,0,0,0,100,0,0,100,0,100,...,0,0,100,0,100,0,100,0,0,100
2021-10-04,0,0,0,0,100,0,0,100,0,100,...,0,0,100,0,100,0,100,0,0,100
2021-10-05,0,0,0,0,100,0,0,100,0,100,...,0,0,100,0,100,0,100,0,0,100
2021-10-06,0,0,0,0,100,0,0,100,0,100,...,0,0,100,0,100,0,100,0,0,100
2021-10-07,0,0,0,0,100,0,0,100,0,100,...,0,0,0,0,0,0,0,0,0,100
2021-10-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2021-10-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-10-12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-10-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## TD3

Test TD3, predict

In [77]:
trained_model = trained_td3
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_model, 
    environment = e_trade_gym)

el modelo es 
estos son los test obs
[[1.0000000e+06 1.4065350e+02 1.9582225e+02 1.6772746e+02 2.2600000e+02
  1.8351404e+02 2.7490091e+02 5.0711670e+01 9.3807129e+01 1.7543822e+02
  3.5233066e+02 3.0777103e+02 2.0362247e+02 1.2042436e+02 5.0125824e+01
  1.4843318e+02 1.5452452e+02 4.8765842e+01 2.2827776e+02 1.2634029e+02
  7.5069099e+01 2.8285742e+02 1.4231578e+02 1.3033842e+02 1.4367572e+02
  3.7738495e+02 2.2568965e+02 4.5867676e+01 4.0303814e+01 4.3821529e+01
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.4449200e+02
  2.1555800e+02 1.7530000e+02 2.2719800e+02 1.9995599e+02 2.7714801e+02
  5.5990002e+01 1.0418000e+

Store graph of the value of portfolio

In [78]:
df_account_value_td3['date'] = pd.to_datetime(df_account_value_td3['date']).dt.normalize()
plt.figure(figsize=(14, 7))
plt.plot(df_account_value_td3['date'], df_account_value_td3['account_value'])

# Setting major locator
locator = mdates.MonthLocator(interval=1)  # Show a tick every week
plt.gca().xaxis.set_major_locator(locator)

# Setting formatter
formatter = mdates.DateFormatter('%Y-%m-%d')
plt.gca().xaxis.set_major_formatter(formatter)

# Optionally, auto-format date labels to prevent overlap
plt.gcf().autofmt_xdate()

plt.title('Account Value Testing for TD3')
plt.xlabel('Date')
plt.ylabel('Account Value')
plt.grid(True)

plt.savefig(f'resultadosTFG/set_1-/td3/testing/accountValue.png')
plt.close()


Trades made in testing

In [79]:
df_actions_td3.head(30)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2021-10-01,0,100,0,100,100,0,0,100,100,100,...,0,0,100,0,0,0,0,0,0,100
2021-10-04,0,100,0,100,100,0,0,100,100,100,...,0,0,100,0,0,0,0,0,0,100
2021-10-05,0,100,0,100,100,0,0,100,100,100,...,0,0,100,0,0,0,0,0,0,100
2021-10-06,0,100,0,100,100,0,0,100,100,100,...,0,0,100,0,0,0,0,0,0,100
2021-10-07,0,0,0,0,54,0,0,100,100,100,...,0,0,100,0,0,0,0,0,0,100
2021-10-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2021-10-11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-10-12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-10-13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## SAC

Test SAC, predict

Account value is the DF that stores the history of the value of the assets (portfolio value)

In [80]:
trained_model = trained_sac
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_model, 
    environment = e_trade_gym)

el modelo es 
estos son los test obs
[[1.0000000e+06 1.4065350e+02 1.9582225e+02 1.6772746e+02 2.2600000e+02
  1.8351404e+02 2.7490091e+02 5.0711670e+01 9.3807129e+01 1.7543822e+02
  3.5233066e+02 3.0777103e+02 2.0362247e+02 1.2042436e+02 5.0125824e+01
  1.4843318e+02 1.5452452e+02 4.8765842e+01 2.2827776e+02 1.2634029e+02
  7.5069099e+01 2.8285742e+02 1.4231578e+02 1.3033842e+02 1.4367572e+02
  3.7738495e+02 2.2568965e+02 4.5867676e+01 4.0303814e+01 4.3821529e+01
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.4449200e+02
  2.1555800e+02 1.7530000e+02 2.2719800e+02 1.9995599e+02 2.7714801e+02
  5.5990002e+01 1.0418000e+

Store graph of the value of portfolio

In [81]:
df_account_value_sac['date'] = pd.to_datetime(df_account_value_sac['date']).dt.normalize()
plt.figure(figsize=(14, 7))
plt.plot(df_account_value_sac['date'], df_account_value_sac['account_value'])

# Setting major locator
locator = mdates.MonthLocator(interval=1)  # Show a tick every week
plt.gca().xaxis.set_major_locator(locator)

# Setting formatter
formatter = mdates.DateFormatter('%Y-%m-%d')
plt.gca().xaxis.set_major_formatter(formatter)

# Optionally, auto-format date labels to prevent overlap
plt.gcf().autofmt_xdate()

plt.title('Account Value Testing for SAC')
plt.xlabel('Date')
plt.ylabel('Account Value')
plt.grid(True)

plt.savefig(f'resultadosTFG/set_1-/sac/testing/accountValue.png')
plt.close()


Trades made in testing

In [82]:
df_actions_sac.head(30)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
date,,,,,,,,,,,,,,,,,,,,,
2021-10-01,84,24,0,0,64,70,52,0,0,0,...,90,78,81,88,0,23,90,0,67,0
2021-10-04,84,24,0,0,64,70,52,0,0,0,...,90,78,81,88,0,23,90,0,67,0
2021-10-05,84,24,0,0,64,70,52,0,0,0,...,90,78,81,88,0,23,90,0,67,0
2021-10-06,84,24,0,0,64,70,52,0,0,0,...,90,78,81,88,0,23,90,0,67,0
2021-10-07,84,0,0,0,64,70,52,0,0,0,...,90,78,81,88,0,0,90,0,67,0
2021-10-08,26,-89,0,0,-63,92,1,0,0,90,...,99,0,0,0,0,-47,0,0,0,0
2021-10-11,0,-7,0,0,-74,95,0,0,0,61,...,99,0,0,1,0,-45,0,0,0,0
2021-10-12,0,0,0,0,-83,97,1,0,0,2,...,99,0,0,0,0,0,0,0,0,0
2021-10-13,0,0,0,0,-88,93,0,0,0,0,...,99,0,0,0,0,0,0,0,0,0


Obtain results to then merge: merge in one DF the Portfolio Value given by each algorithm

In [83]:
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
df_result_a2c.rename(columns = {'account_value':'a2c'}, inplace = True)
df_result_ddpg = df_account_value_ddpg.set_index(df_account_value_ddpg.columns[0])
df_result_ddpg.rename(columns = {'account_value':'ddpg'}, inplace = True)
df_result_td3 = df_account_value_td3.set_index(df_account_value_td3.columns[0])
df_result_td3.rename(columns = {'account_value':'td3'}, inplace = True)
df_result_ppo = df_account_value_ppo.set_index(df_account_value_ppo.columns[0])
df_result_ppo.rename(columns = {'account_value':'ppo'}, inplace = True)
df_result_sac = df_account_value_sac.set_index(df_account_value_sac.columns[0])
df_result_sac.rename(columns = {'account_value':'sac'}, inplace = True)

We can also get stats from each stock individually (even though the analysis won't focus on that)

In [84]:
#baseline stats

print("==============Get Baseline Stats===========")
print("==============Baseline Stats of Profitability from DJIA===========")
print("This is the real performance of the index")
df_dji_ = get_baseline(
        ticker="^DJI", 
        start = TEST_START_DATE,
        end = TEST_END_DATE)
stats = backtest_stats(df_dji_, value_col_name = 'close')

==============Get Baseline Stats===========
==============Baseline Stats of Profitability from DJIA===========
This is the real performance of the index


[*********************100%%**********************]  1 of 1 completed

Shape of DataFrame:  (354, 8)
Annual return         -0.034876
Cumulative returns    -0.048644
Annual volatility      0.181612
Sharpe ratio          -0.105351
Calmar ratio          -0.158953
Stability              0.280983
Max drawdown          -0.219408
Omega ratio            0.982546
Sortino ratio         -0.146974
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.970602
Daily value at risk   -0.022957
dtype: float64


get_baseline vuelve a fetchear los stocks de YahooFinance dadas esas fechas.

backtest_stats llama a get daily return y luego saca stats de esa Serie con metodo que pide:

    """
    Calculates various performance metrics of a strategy, for use in
    plotting.show_perf_stats.

    Parameters
    ----------
    returns : pd.Series
        Daily returns of the strategy, noncumulative.
         - See full explanation in tears.create_full_tear_sheet.
    factor_returns : pd.Series, optional
        Daily noncumulative returns of the benchmark factor to which betas are
        computed. Usually a benchmark such as market returns.
         - This is in the same style as returns.
         - If None, do not compute alpha, beta, and information ratio.
    positions : pd.DataFrame
        Daily net position values.
         - See full explanation in tears.create_full_tear_sheet.
    transactions : pd.DataFrame
        Prices and amounts of executed trades. One row per trade.
        - See full explanation in tears.create_full_tear_sheet.
    turnover_denom : str
        Either AGB or portfolio_value, default AGB.
        - See full explanation in txn.get_turnover.

    Returns
    -------
    pd.Series
        Performance metrics.
    """

get daily profitability * invested amount = portfolio value in case of investing 1000000$ on DJ and not touching it

In [85]:
df_dji = pd.DataFrame()
df_dji['date'] = df_account_value_a2c['date']
df_dji['account_value'] = df_dji_['close'] / df_dji_['close'][0] * env_kwargs["initial_amount"] # get daily profitability * invested amount = portfolio value in case of investing 1000000$ on DJ and not touching it
#df_dji.to_csv("df_dji.csv")
df_dji = df_dji.set_index(df_dji.columns[0])
#df_dji.to_csv("df_dji+.csv")

In [86]:
df_dji.rename(columns = {'account_value':'dji'}, inplace = True)

In [87]:
df_result_td3

,td3
date,
2021-10-01,1.000000e+06
2021-10-04,9.984152e+05
2021-10-05,1.002491e+06
2021-10-06,1.002842e+06
2021-10-07,1.008457e+06
...,...
2023-02-21,9.786110e+05
2023-02-22,9.737047e+05
2023-02-23,9.765710e+05


Compare DJIA with the rest of the algorithms

In [88]:
result = pd.DataFrame()
result = pd.merge(result, df_result_a2c, how='outer', left_index=True, right_index=True)
result = pd.merge(result, df_result_ddpg, how='outer', left_index=True, right_index=True)
result = pd.merge(result, df_result_td3, how='outer', left_index=True, right_index=True)
result = pd.merge(result, df_result_ppo, how='outer', left_index=True, right_index=True)
result = pd.merge(result, df_result_sac, how='outer', left_index=True, right_index=True)
result = pd.merge(result, df_dji, how='outer', left_index=True, right_index=True)
display(result)

,a2c,ddpg,td3,ppo,sac,dji
date,,,,,,
2021-10-01,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06,1.000000e+06
2021-10-04,9.978582e+05,9.979904e+05,9.984152e+05,9.996388e+05,9.978001e+05,9.905746e+05
2021-10-05,1.000081e+06,1.002659e+06,1.002491e+06,1.000371e+06,1.000274e+06,9.996566e+05
2021-10-06,1.001398e+06,1.004695e+06,1.002842e+06,1.000640e+06,1.003028e+06,1.002637e+06
2021-10-07,1.008849e+06,1.014824e+06,1.008457e+06,1.001789e+06,1.011014e+06,1.012483e+06
...,...,...,...,...,...,...
2023-02-21,1.039031e+06,1.024499e+06,9.786110e+05,8.853356e+05,9.143135e+05,9.651327e+05
2023-02-22,1.035208e+06,1.021529e+06,9.737047e+05,8.859647e+05,9.153385e+05,9.626710e+05
2023-02-23,1.038890e+06,1.024594e+06,9.765710e+05,8.887933e+05,9.187608e+05,9.658412e+05


Save and Plot!

In [89]:
plt.rcParams["figure.figsize"] = (15,5)
plt.figure();
result.plot();
plt.savefig(f'resultadosTFG/set_1-/AlgorithmsVsDJI.png')
plt.close()